In [5]:
import pandas as pd

Começamos baixando o arquivo de dicionario disponibilizado pelo ime-usp:

https://www.ime.usp.br/~pf/dicios/br-latin1.txt

Filtramos as palavras com apenas 5 letras e retiramos todas as acentuações

In [2]:
import unicodedata
import re

def remove_acentos(string):
    normalized = unicodedata.normalize('NFD', string)
    return re.sub(r'[\u0300-\u036f]', '', normalized).casefold()

In [3]:
palavras = []
file = open('br-utf8.txt', 'r', encoding = 'utf8')

while True:
    line = file.readline()
    palavra = line.strip().replace('\n','')
    if not line:
        break
    if len(palavra) == 5:
        #print(palavra)
        palavras.append(remove_acentos(palavra))
        

file.close()

In [10]:
df = pd.DataFrame(palavras)
df.drop_duplicates(inplace = True)
df.rename(columns = {0:'palavra'}, inplace = True)
df.head()

,palavra
0,aarao
1,abner
2,acaia
3,acker
4,aires


In [11]:
def conta_vogais(palavra):
    
    contador = 0
    vogais = 'aeiouAEIOU'
    allowed = frozenset('aeiouAEIOU')
    
    for letra in palavra:
        if letra in vogais:
            contador += 1
    vogais_unicas = len(allowed.intersection(palavra))
    
    return vogais_unicas, contador


conta_vogais('perde')    



(1, 2)

In [12]:
df['n_vogais'] = df['palavra'].apply(lambda x: conta_vogais(x)[1])
df['vogais_unicas'] = df['palavra'].apply(lambda x: conta_vogais(x)[0])
df

,palavra,n_vogais,vogais_unicas
0,aarao,4,2
1,abner,2,2
2,acaia,4,2
3,acker,2,2
4,aires,3,3
...,...,...,...
6021,unica,3,3
6022,unico,3,3
6023,urico,3,3
6024,uteis,3,3


No inicio iria só trabalhar só com palavras e vogais, depois pesquisando como trabalhar com string acabei decidingo em separar todas as letras das palavras por coluna

gerando assim um dataframe com as colunas:

- palavra
- numero de vogais
- vogais unicas
- letra 1
- letra 2
- letra 3
- letra 4
- letra 5

porém trabalhei com posição da letra coemçando em 0

In [13]:
df['letra_0'] = df.palavra.str[0]
df['letra_1'] = df.palavra.str[1]
df['letra_2'] = df.palavra.str[2]
df['letra_3'] = df.palavra.str[3]
df['letra_4'] = df.palavra.str[4]

# Melhora na escolha de palavra

usando um pouco dos conceitos de NLP para carregar as palavras de alguns livros disponibilizados publicamente:
- Dom Casmurro
- D. Quixote
- Iracema
- Os Sertoes
- Primo Basilio
- Poemas Fernando Pessoa
    
Com esses livros tentei gerar uma variavel de probabilidade das palavras mais utilizadas para assim filtrar pela com maior probabilidade de uso


In [14]:
import nltk
local = 'livros.txt'
def abrir_arquivo(local):
    with open(local, 'r', encoding = 'utf8') as f:
        palavras = f.read()
    f.close()
    lista_tokens = nltk.tokenize.word_tokenize(palavras)
    lista_palavras = []
    for i in lista_tokens:
        if i.isalpha():
            if len(i) == 5:
                lista_palavras.append(remove_acentos(i.lower()))
            
    f.close()
    return lista_palavras

lista_palavras = abrir_arquivo(local)
lista_palavras

['carro',
 'parte',
 'desta',
 'assim',
 'ainda',
 'quase',
 'falar',
 'coisa',
 'abrir',
 'ainda',
 'enfim',
 'baeta',
 'verde',
 'senao',
 'mumia',
 'foram',
 'muito',
 'saude',
 'conta',
 'muito',
 'juizo',
 'muito',
 'razao',
 'modos',
 'abuso',
 'outro',
 'solon',
 'senao',
 'outra',
 'falou',
 'tanta',
 'todos',
 'juizo',
 'tocar',
 'falsa',
 'assim',
 'lance',
 'lance',
 'vindo',
 'corte',
 'entre',
 'disse',
 'certo',
 'turco',
 'sabia',
 'nuvem',
 'temor',
 'quase',
 'todos',
 'malta',
 'tempo',
 'estas',
 'horas',
 'muito',
 'longe',
 'ouviu',
 'estas',
 'disse',
 'entre',
 'pobre',
 'mesmo',
 'lista',
 'disse',
 'muito',
 'outra',
 'coisa',
 'senao',
 'todos',
 'maior',
 'parte',
 'reino',
 'facil',
 'justo',
 'breve',
 'caber',
 'vossa',
 'merce',
 'tarda',
 'disse',
 'agora',
 'outro',
 'minha',
 'vossa',
 'merce',
 'roque',
 'homem',
 'algum',
 'disse',
 'disse',
 'tenho',
 'homem',
 'fosse',
 'disse',
 'neste',
 'pagar',
 'vossa',
 'merce',
 'minha',
 'corpo',
 'certo',


In [15]:
def probabilidade(palavra):
    return frequencia[palavra]/total_de_palavras

frequencia = nltk.FreqDist(lista_palavras)
total_de_palavras = len(lista_palavras)
frequencia.most_common(10)

[('minha', 847),
 ('todos', 813),
 ('disse', 800),
 ('ainda', 788),
 ('muito', 742),
 ('assim', 716),
 ('sobre', 616),
 ('vossa', 590),
 ('entre', 589),
 ('mesmo', 526)]

In [16]:
def conta_consoantes(palavra):
    
    contador = 0
    vogais = 'bcdfghjklmnpqrstvxywz'
    allowed = frozenset(vogais)
    
    for letra in palavra:
        if letra in vogais:
            contador += 1
    consoantes_unicas = len(allowed.intersection(palavra))
    
    return consoantes_unicas, contador

In [17]:
df['probabilidade'] = df.palavra.apply(lambda x: probabilidade(x))
df['n_consoantes'] = df.palavra.apply(lambda x: conta_consoantes(x)[1])
df['consoantes_unicas'] = df.palavra.apply(lambda x: conta_consoantes(x)[0])



In [18]:
from collections import OrderedDict

def tira_duplicadas_palavra(s): #removendo as duplicadas da string
    d = OrderedDict()
    for c in s:
        if c not in d:
            d[c] = 0
        d[c] += 1

    return ''.join(d.keys())

def letras_diferentes(palavra):
    return len(tira_duplicadas_palavra(palavra))

In [19]:
df['n_letras_diferentes'] = df.palavra.apply(lambda x: letras_diferentes(x))

In [21]:
df.head()

,palavra,n_vogais,vogais_unicas,letra_0,letra_1,letra_2,letra_3,letra_4,probabilidade,n_consoantes,consoantes_unicas,n_letras_diferentes
0,aarao,4,2,a,a,r,a,o,0.000000,1,1,3
1,abner,2,2,a,b,n,e,r,0.000000,3,3,5
2,acaia,4,2,a,c,a,i,a,0.000000,1,1,3
3,acker,2,2,a,c,k,e,r,0.000000,3,3,5
4,aires,3,3,a,i,r,e,s,0.000037,2,2,5


O Dataframe acima é a versão final utilizada para resolução do term.ooo

-----------------------------

Palavras excluidas do dicionario baixado da usp:
- lermo
- canaa
- propo
- coemo
- doemo
- jobim
- suamo
- sorta
- premo
- iremo
- prepo

-----------------------------